Dosyaları 2 saniyelere ayırma

In [9]:
from pydub import AudioSegment
import os

# Tam WAV dosyalarının bulunduğu dizin
wav_dir = r"C:\Users\Mevlit\Desktop\yazilim_sinama\sesler-wav"
# Parçalı WAV dosyalarının kaydedileceği dizin
parcali_wav_dir = r"C:\Users\Mevlit\Desktop\yazilim_sinama\parcali-sesler-wav"

# WAV dosyalarının listesi
wav_files = os.listdir(wav_dir)

# Her WAV dosyasını işleyin
for wav_file in wav_files:
    # WAV dosyasını yükle
    audio = AudioSegment.from_wav(os.path.join(wav_dir, wav_file))
    # 2 saniyelik parçalara böl
    chunks = audio[::2000]
    
    # Parçaları kaydet
    for i, chunk in enumerate(chunks):
        chunk.export(os.path.join(parcali_wav_dir, f"{wav_file[:-4]}_{i}.wav"), format="wav")

MFCC özelliklerini çıkarmak

Veri Ön İşleme ve Özellik Çıkarımı

In [10]:
import os
import librosa
import numpy as np

# Ses dosyalarının bulunduğu dizin
ses_dizin = r'C:\Users\Mevlit\Desktop\yazilim_sinama\parcali-sesler-wav'

# MFCC özelliklerinin kaydedileceği dizin
mfcc_dizin = r'C:\Users\Mevlit\Desktop\yazilim_sinama\parcali-sesler-mfcc'

# MFCC parametreleri
n_mfcc = 128

# Framelere ayırma parametreleri
frame_length = 25  # milisaniye cinsinden
frame_stride = 10   # milisaniye cinsinden

# Dosyaları işleme
for dosya_adı in os.listdir(ses_dizin):
    if dosya_adı.endswith('.wav'):
        dosya_yolu = os.path.join(ses_dizin, dosya_adı)
        ses, sr = librosa.load(dosya_yolu, sr=None)
        mfcc = librosa.feature.mfcc(y=ses, sr=sr, n_mfcc=n_mfcc, hop_length=int(frame_stride * sr / 1000),
                                     n_fft=int(frame_length * sr / 1000))
        mfcc_dosya_adı = dosya_adı.split('.')[0] + '.npy'
        mfcc_dosya_yolu = os.path.join(mfcc_dizin, mfcc_dosya_adı)
        np.save(mfcc_dosya_yolu, mfcc)

print("MFCC özellikleri başarıyla oluşturuldu.")
print(mfcc.shape)

MFCC özellikleri başarıyla oluşturuldu.
(128, 201)


Modeli eğitmek

In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import os
import joblib

# MFCC özelliklerinin bulunduğu dizin
mfcc_dizin = r'C:\Users\Mevlit\Desktop\yazilim_sinama\parcali-sesler-mfcc'

X = []
y = []

# MFCC dosyalarını yükleme
for dosya_adı in os.listdir(mfcc_dizin):
    if dosya_adı.endswith('.npy'):
        dosya_yolu = os.path.join(mfcc_dizin, dosya_adı)
        mfcc = np.load(dosya_yolu)
        X.append(np.mean(mfcc, axis=1))  # Her dosya için ortalama MFCC vektörü
        y.append(dosya_adı.split('_')[0])  # Dosya adından etiket çıkarma

X = np.array(X)
y = np.array(y)

# Etiketleri sayısal değerlere dönüştürme
le = LabelEncoder()
y = le.fit_transform(y)

# Veri kümesini eğitim ve test kümelerine ayırma
X_egitim, X_test, y_egitim, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# MLP modeli oluşturma ve eğitme
model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
model.fit(X_egitim, y_egitim)

# Modelin doğruluğunu değerlendirme
dogruluk = model.score(X_test, y_test)
print(f"Model doğruluğu: {dogruluk}")

print(X_egitim.shape)

# Modeli diske kaydetme
model_kayit_yolu = r'C:\Users\Mevlit\Desktop\yazilim_sinama\model-yazilimsinama.pkl'
joblib.dump(model, model_kayit_yolu)

Model doğruluğu: 0.9753694581280788
(809, 128)


In [13]:
from sklearn.metrics import classification_report, confusion_matrix

# Test veri seti üzerinde tahmin yapma
tahminler = model.predict(X_test)

# Sınıflandırma raporu ve karışıklık matrisini yazdırma
print("Sınıflandırma Raporu:")
print(classification_report(y_test, tahminler))

print("Karışıklık Matrisi:")
print(confusion_matrix(y_test, tahminler))

Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        21
           1       0.94      0.94      0.94        16
           2       0.95      1.00      0.98        21
           3       1.00      1.00      1.00        21
           4       1.00      1.00      1.00        21
           5       0.96      1.00      0.98        22
           6       1.00      0.81      0.89        21
           7       0.90      1.00      0.95        19
           8       1.00      1.00      1.00        19
           9       1.00      1.00      1.00        22

    accuracy                           0.98       203
   macro avg       0.98      0.97      0.97       203
weighted avg       0.98      0.98      0.97       203

Karışıklık Matrisi:
[[21  0  0  0  0  0  0  0  0  0]
 [ 0 15  1  0  0  0  0  0  0  0]
 [ 0  0 21  0  0  0  0  0  0  0]
 [ 0  0  0 21  0  0  0  0  0  0]
 [ 0  0  0  0 21  0  0  0  0  0]
 [ 0  0  0  0  0 22  0  0  0  0]
 

Uygulama Geliştirme - Dosya

In [17]:
import numpy as np
import os
import joblib

# Eğitilmiş modeli yükleme
model_kayit_yolu = r'C:\Users\Mevlit\Desktop\yazilim_sinama\model-yazilimsinama.pkl'
model = joblib.load(model_kayit_yolu)

# Test edilecek ses dosyasının MFCC özelliklerini yükleme
test_ses_dosyasi = r'C:\Users\Mevlit\Desktop\yazilim_sinama\parcali-sesler-mfcc\ozcan_61.npy'
mfcc = np.load(test_ses_dosyasi)

# Model üzerinden tahmin yapma
tahmin = model.predict(np.mean(mfcc, axis=1).reshape(1, -1))

# Tahmini sonuç
print("Tahmin edilen kişi: ", tahmin)

Tahmin edilen kişi:  [7]


Uygulama Geliştirme - Mikrofon

Canlı ses kaydı güncel

In [11]:
import numpy as np
import sounddevice as sd
import joblib
import librosa
import soundfile as sf
import os

# Eğitilmiş modeli yükleme
model_kayit_yolu = r'C:\Users\Mevlit\Desktop\yazilim_sinama\model-yazilimsinama.pkl'
model = joblib.load(model_kayit_yolu)

# WAV dosyalarının bulunduğu klasör
wav_dosyasi_klasoru = r'C:\Users\Mevlit\Desktop\yazilim_sinama\sesler-wav'

# .wav uzantılı dosyaların isimlerini al
sinif_isimleri = [
    os.path.splitext(dosya)[0] for dosya in os.listdir(wav_dosyasi_klasoru) if dosya.endswith(".wav")
]

# Mikrofondan ses almak için gerekli parametreler
saniye_basina_ornek = 44100  # Örnekleme hızı
kanal_sayisi = 1  # Tek kanallı ses
parca_suresi = 5  # Parça başına süre (saniye)
parca_ornek = saniye_basina_ornek * parca_suresi  # 5 saniyelik örnek boyutu

# "yazilim_sinama" dizini içinde "test" klasörü oluştur
kayit_dizini = r"C:\Users\Mevlit\Desktop\yazilim_sinama\test"  # "test" klasörü yolu
os.makedirs(kayit_dizini, exist_ok=True)  # Klasörü oluştur (zaten varsa hata vermez)

# Tüm ses kaydı için dosya yolu
tum_kayit_yolu = os.path.join(kayit_dizini, "tum_kayit.wav")

print("Gerçek zamanlı ses kaydı başlıyor...")
parca_sayaci = 0

# Tüm kaydı tek bir dosyada saklamak için SoundFile dosyası aç
with sf.SoundFile(tum_kayit_yolu, mode='w', samplerate=saniye_basina_ornek, channels=kanal_sayisi) as tum_dosya:
    with sd.InputStream(samplerate=saniye_basina_ornek, channels=kanal_sayisi, dtype='float32') as stream:
        for _ in range(0, 60, parca_suresi):
            data = stream.read(parca_ornek)[0]  # 5 saniyelik veriyi oku
            tum_dosya.write(data)  # Tüm kaydı saklamak için dosyaya yaz
            parca_sayaci += 1

            # Her parçayı ayrı dosya olarak kaydet
            dosya_adi = os.path.join(kayit_dizini, f"parca_{parca_sayaci}.wav")
            sf.write(dosya_adi, data, samplerate=saniye_basina_ornek)
            print(f"Parça kaydedildi: {dosya_adi}")

            # Parçayı işlemek için tahmin işlemi
            y, sr = librosa.load(dosya_adi, sr=saniye_basina_ornek)
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=128)
            mfcc = np.mean(mfcc.T, axis=0)  # Ortalama MFCC vektörü
            
            # Model üzerinden tahmin yapma
            tahmin_indeksi = model.predict(mfcc.reshape(1, -1))[0]
            tahmin_isim = sinif_isimleri[tahmin_indeksi]
            
            # Tahmini sonuç
            print("Tahmin edilen kişi: ", tahmin_isim)

print("Tüm kayıt işlemi tamamlandı ve tüm ses kaydedildi!")


Gerçek zamanlı ses kaydı başlıyor...
Parça kaydedildi: C:\Users\Mevlit\Desktop\yazilim_sinama\test\parca_1.wav
Tahmin edilen kişi:  ozcan
Parça kaydedildi: C:\Users\Mevlit\Desktop\yazilim_sinama\test\parca_2.wav
Tahmin edilen kişi:  hasan
Parça kaydedildi: C:\Users\Mevlit\Desktop\yazilim_sinama\test\parca_3.wav
Tahmin edilen kişi:  ozcan
Parça kaydedildi: C:\Users\Mevlit\Desktop\yazilim_sinama\test\parca_4.wav
Tahmin edilen kişi:  ozcan
Parça kaydedildi: C:\Users\Mevlit\Desktop\yazilim_sinama\test\parca_5.wav
Tahmin edilen kişi:  ozcan
Parça kaydedildi: C:\Users\Mevlit\Desktop\yazilim_sinama\test\parca_6.wav
Tahmin edilen kişi:  ozcan
Parça kaydedildi: C:\Users\Mevlit\Desktop\yazilim_sinama\test\parca_7.wav
Tahmin edilen kişi:  ozcan
Parça kaydedildi: C:\Users\Mevlit\Desktop\yazilim_sinama\test\parca_8.wav
Tahmin edilen kişi:  ozcan
Parça kaydedildi: C:\Users\Mevlit\Desktop\yazilim_sinama\test\parca_9.wav
Tahmin edilen kişi:  ozcan
Parça kaydedildi: C:\Users\Mevlit\Desktop\yazilim_sina

Transkript çıkarma

In [12]:
import os
from google.cloud import speech
from google.cloud import storage

# Google Cloud kimlik dosyasını ayarlayın
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\Mevlit\Desktop\yazilim_sinama\yazilimsinama-0648-ea2747e0ecd7.json"

def ses_dosyasini_yukle(bucket_name, source_file_name, destination_blob_name):
    """Ses dosyasını Google Cloud Storage'a yükler."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)
    print(f"{source_file_name} yüklendi ve Google Cloud Storage'da {destination_blob_name} olarak saklandı.")
    return f"gs://{bucket_name}/{destination_blob_name}"

def transkript_cikart_uzun_ses(audio_uri):
    """Google Speech-to-Text ile uzun ses dosyasının transkriptini çıkarır."""
    client = speech.SpeechClient()

    audio = speech.RecognitionAudio(uri=audio_uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=44100,
        language_code="tr-TR"
    )

    # Uzun ses dosyasını işleme
    operation = client.long_running_recognize(config=config, audio=audio)
    print("Ses dosyası işleniyor, lütfen bekleyin...")
    response = operation.result(timeout=600)

    # Transkripti birleştirme
    transkript = ""
    for result in response.results:
        transkript += result.alternatives[0].transcript + "\n"
    return transkript

# Ses dosyasının yolu ve Google Cloud Storage ayarları
bucket_name = "ses-kayitlarim"
source_file_name = r"C:\Users\Mevlit\Desktop\yazilim_sinama\test\tum_kayit.wav"
destination_blob_name = "tum_kayit.wav"

# Ses dosyasını yükleyip URI al
audio_uri = ses_dosyasini_yukle(bucket_name, source_file_name, destination_blob_name)

# Transkript oluştur ve ekrana yazdır
transkript = transkript_cikart_uzun_ses(audio_uri)
print("Transkript:")
print(transkript)

# Transkripti bir dosyaya kaydet
transkript_dosyasi = r"C:\Users\Mevlit\Desktop\yazilim_sinama\test\transkript.txt"
with open(transkript_dosyasi, "w", encoding="utf-8") as f:
    f.write(transkript)
print(f"Transkript kaydedildi: {transkript_dosyasi}")


C:\Users\Mevlit\Desktop\yazilim_sinama\test\tum_kayit.wav yüklendi ve Google Cloud Storage'da tum_kayit.wav olarak saklandı.
Ses dosyası işleniyor, lütfen bekleyin...
Transkript:
Merhaba arkadaşlar ben Murat Bugün sizlerle birlikte yepyeni bir video sesli yapıyoruz Özcan beni tanıyorsun Neyse bugün sana bir şeyler anlatayım Bugün ben Hasan değilim Hasan daha Samsun'da bugün rüyamda Hiçbir şey görmedim arkadaşlar Ben bugün rüyamda bulamadım Onun da şey yok şu an kafam ağrıyor yine okula Sen ne yapacağım bilmiyorum ne yapacağım bilmiyorum Kaldım öyle beni doğru tahmin ediyorsun geleceğim ama Yahu acaba bu sikeceğim 2 cm nasıl tarz Büyükçekmece konu merak ettiğim için böyle şeyler dedim arkadaşlar normale terbiyeli bir çocuğum onun dışından başka diyecek bir şeyim yok gibi aklıma bir şey gelmiyor neyse ya güne çok bol hakkında konuşalım Şenol Güneş eklerine arkadaşlar

Transkript kaydedildi: C:\Users\Mevlit\Desktop\yazilim_sinama\test\transkript.txt


Konu analizi

In [16]:
from deep_translator import GoogleTranslator
from google.cloud import language_v1

def google_konu_analizi():
    # Transkripti dosyadan oku
    transkript_dosyasi = r"C:\Users\Mevlit\Desktop\yazilim_sinama\test\transkript.txt"
    with open(transkript_dosyasi, "r", encoding="utf-8") as f:
        transkript = f.read()

    # Deep Translator ile metni çevir
    translated = GoogleTranslator(source='tr', target='en').translate(transkript)
    print(f"Çevrilmiş Metin: {translated}")
    
    # Google Cloud Natural Language API ile analiz
    client = language_v1.LanguageServiceClient()
    document = language_v1.Document(content=translated, type_=language_v1.Document.Type.PLAIN_TEXT)
    response = client.analyze_entities(document=document)

    # Konu analizi sonuçları
    print("Konu Analizi Sonuçları:")
    for entity in response.entities:
        print(f"Ad: {entity.name}, Tür: {entity.type_.name}, Önem Skoru: {entity.salience:.2f}")

# Analiz yap
google_konu_analizi()


Çevrilmiş Metin: Hello friends, today I will talk about artificial intelligence technologies.

As you know, artificial intelligence has become very popular in recent years and has begun to be used in many sectors.

For example, in the healthcare sector, it helps doctors in the diagnosis process.

In education, it provides personalized learning experiences and enables students to work more efficiently.
Konu Analizi Sonuçları:
Ad: friends, Tür: PERSON, Önem Skoru: 0.44
Ad: artificial intelligence technologies, Tür: OTHER, Önem Skoru: 0.18
Ad: sectors, Tür: OTHER, Önem Skoru: 0.07
Ad: artificial intelligence, Tür: OTHER, Önem Skoru: 0.07
Ad: healthcare sector, Tür: OTHER, Önem Skoru: 0.06
Ad: example, Tür: OTHER, Önem Skoru: 0.05
Ad: education, Tür: OTHER, Önem Skoru: 0.04
Ad: learning experiences, Tür: OTHER, Önem Skoru: 0.03
Ad: students, Tür: PERSON, Önem Skoru: 0.02
Ad: doctors, Tür: PERSON, Önem Skoru: 0.02
Ad: diagnosis process, Tür: OTHER, Önem Skoru: 0.02


Yeni kişi ekleme

In [26]:
import numpy as np
import sounddevice as sd
import joblib
import librosa
import soundfile as sf
import os
import tkinter as tk
from tkinter import simpledialog

# Mikrofondan ses almak için gerekli parametreler
saniye_basina_ornek = 44100  # Örnekleme hızı (örneğin, 44100 Hz)
saniye = 90  # 90 saniyelik ses al
kanal_sayisi = 1  # Tek kanallı ses

kisi_adi = simpledialog.askstring("Yeni Kişi", "Yeni kişinin adını girin:")#Kişinin adını alma

print("Konuşmaya başlayın")
ses = sd.rec(int(saniye_basina_ornek * saniye), samplerate=saniye_basina_ornek, channels=kanal_sayisi, dtype='float32')
sd.wait()


# Ses dosyasını WAV olarak kaydetme
kayit_yolu = fr'C:\Users\Mevlit\Desktop\yazilim_sinama\sesler-wav\{kisi_adi}.wav'

sf.write(kayit_yolu, np.squeeze(ses), saniye_basina_ornek)
print("Sesiniz başarıyla kaydedildi.")

#Kaydedilen WAV dosyasını 2 saniyelik parçalara bölmek ve kaydetmek
wav_file = fr'C:\Users\Mevlit\Desktop\yazilim_sinama\sesler-wav\{kisi_adi}.wav'
wav_dir = r'C:\Users\Mevlit\Desktop\yazilim_sinama\parcali-sesler-wav' #Parcalanmıs dosyaların kaydedileceği dizin 
audio = AudioSegment.from_wav(wav_file) # WAV dosyasını yükle
chunks = audio[::2000]
for i, chunk in enumerate(chunks):
        chunk.export(os.path.join(wav_dir, f"{kisi_adi}_{i}.wav"), format="wav")
print("bölme işlemi tamamlandı")
# Kaydedilen kişinin MFCC özelliklerini çıkarmak

# Ses dosyalarının bulunduğu dizin
ses_dizin = r'C:\Users\Mevlit\Desktop\yazilim_sinama\parcali-sesler-wav'

# MFCC özelliklerinin kaydedileceği dizin
mfcc_dizin = r'C:\Users\Mevlit\Desktop\yazilim_sinama\parcali-sesler-mfcc'

# MFCC parametreleri
n_mfcc = 128

# Framelere ayırma parametreleri
frame_length = 25  # milisaniye cinsinden
frame_stride = 10   # milisaniye cinsinden

for dosya_adı in os.listdir(ses_dizin):
    if dosya_adı.startswith(kisi_adi):
        dosya_yolu = os.path.join(ses_dizin, dosya_adı)
        ses, sr = librosa.load(dosya_yolu, sr=None)
        mfcc = librosa.feature.mfcc(y=ses, sr=sr, n_mfcc=n_mfcc, hop_length=int(frame_stride * sr / 1000),
                                     n_fft=int(frame_length * sr / 1000))
        mfcc_dosya_adı = dosya_adı.split('.')[0] + '.npy'
        mfcc_dosya_yolu = os.path.join(mfcc_dizin, mfcc_dosya_adı)
        np.save(mfcc_dosya_yolu, mfcc)

print("MFCC özellikleri başarıyla oluşturuldu.")

#Modeli güncelleme buraya modeli eğitme ve kaydetme metodunu çağır

Konuşmaya başlayın
Sesiniz başarıyla kaydedildi.
bölme işlemi tamamlandı
MFCC özellikleri başarıyla oluşturuldu.
